In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from pathlib import Path

In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [103]:
def get_AMB_APB(data, channel):
    if channel == 355: 
        config_data_vars = {
            f'LNG_Molecular_Parallel_Attenuated_Backscatter_{channel}':'AMB',
            f'LNG_Particulate_Parallel_Attenuated_Backscatter_{channel}':'APB',
            f'Model_Molecular_Backscatter_{channel}':'MMB',
            f'Model_Molecular_Extinction_{channel}':'MME',
            'Height' : 'Height', 
            'Time' : 'Time'
        }
    else:
        config_data_vars = {
            f'LNG_Total_Attenuated_Backscatter_{channel}':'APB',
            f'Model_Molecular_Backscatter_{channel}':'MMB',
            f'Model_Molecular_Extinction_{channel}':'MME',
            'Height' : 'Height',
            'Time' : 'Time'
        }
#     config_data_vars = [f'LNG_Molecular_Parallel_Attenuated_Backscatter_{channel}', 
#                        f'LNG_Particulate_Parallel_Attenuated_Backscatter_{channel}',
#                        f'Model_Molecular_Backscatter_{channel}',
#                        'Height']
#     print(config_data_vars.keys())
    data = data.where((data['Validity_rate']==1) & (data['LNG_UpDown']==1), drop='True')
    tmp_data = data[list(config_data_vars.keys())]
    tmp_data = tmp_data.rename_vars(config_data_vars)
    return tmp_data


In [104]:
listfiles = sorted(Path('/homedata/nmpnguyen/LNG/NETCDF/').glob('*.nc'))


In [105]:
d = xr.open_dataset(listfiles[0],decode_cf=False)

# d.where((d['Validity_rate']==1) & (d['LNG_UpDown']==1), drop='True')

In [106]:
new_d = get_AMB_APB(d, 355)
new_d 

<xarray.Dataset>
Dimensions:  (height: 3500, time: 815)
Dimensions without coordinates: height, time
Data variables:
    AMB      (time, height) float32 -999.0 -999.0 -999.0 ... -999.0 -999.0
    APB      (time, height) float32 -999.0 -999.0 -999.0 ... -999.0 -999.0
    MMB      (time, height) float32 4.402e-06 4.405e-06 ... 7.88e-06 7.88e-06
    MME      (time, height) float32 3.748e-05 3.751e-05 ... 6.709e-05 6.709e-05
    Height   (time, height) float32 5.871 5.865 5.859 ... -11.3 -11.31 -11.31
    Time     (time, height) float32 2.44e+04 2.44e+04 ... 3.414e+04 3.414e+04
Attributes:
    Level:        L1_TEMP
    Description:  LNG L0 concatenated HSR and DET DATA - LNG data - 532, 1064...
    created:      20220127
    Author:       quitterie.cazenave@latmos.ipsl.fr
    Campaign:     CADDIWA
    Day:          20210908
    Flight:       5
    Thermo:       ERA5
    Angles:       Lidar angles: validated with ground echo

In [123]:
def compute_transmittance_2way(AMB, APB, MMB, MME, alt):
    C = -np.log(AMB/MMB)/2
    print(C.shape)
    E = np.full(C.shape, np.nan)#np.array([[],[]])
    E[:,0] = C[:,0]
    print(E)
    for i in range(1, C.shape[1]):
        E[:,i] = C[:,i] - E[:, i-1]*(alt[:, i]-alt[:, i-1])
    
    # Extinction particular
    MPE = E - MME
    return MPE

In [124]:
mpe = compute_transmittance_2way(new_d['AMB'], new_d['APB'], new_d['MMB'], new_d['MME'], new_d['Height'])

/net/nfs/tools/python-anaconda/3.8/lib/python3.8/site-packages/xarray/core/computation.py:724: RuntimeWarning: invalid value encountered in log
  result_data = func(*input_data)


(815, 3500)
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [122]:
# np.isnan(mpe).all(axis=0)
np.log(-(new_d['AMB'].values / new_d['MMB'].values))/2

array([[9.620082, 9.619758, 9.619428, ..., 9.329177, 9.329177, 9.329177],
       [9.62185 , 9.621565, 9.62128 , ..., 9.329177, 9.329177, 9.329177],
       [9.623712, 9.623419, 9.623133, ..., 9.329177, 9.329177, 9.329177],
       ...,
       [9.729915, 9.729627, 9.72933 , ..., 9.328966, 9.328966, 9.328966],
       [9.73017 , 9.729874, 9.729586, ..., 9.328966, 9.328966, 9.328966],
       [9.730227, 9.729931, 9.729635, ..., 9.328966, 9.328966, 9.328966]],
      dtype=float32)

18.420680743952367